# Parsing av vedtakstekster

Vedtakene i tekstkorpuset må parses, dvs. vi må identifisere strukturen i vedtakene, og dele opp vedtakene i deres ulike deler. Formålet er å analysere korpuset, og å legge til rette for å en variert testing opp mot språkmodellen. Eksempelvis ønsker vi å teste ut semantiske søk der vi bare søker i klagers anførsler. Da kan vi etterpå bruke nemndas vurdering fra de samme vedtakene som kontekst opp mot språkmodellen. Et annet bruksområde er i identifisere vedtak der nemnda deler seg i et flertall og et mindretall. Da er det viktig å først identifisere hvilken del av vedtaket som er nemndas vurdering. 

## Analyse av strukturen i vedtakstekstene
Det ser ut til at mange vedtak har samme struktur: sammendrag, dato, saksnr. og tjenesteyter, så klagers anførsler, tjenesteyters anførsler og til slutt nemndas bemerkninger. Før vi går i gang med selve parsingen, skal vi analysere strukturen vedtakene. Hvilke overskrifter bruker vedtakene, har vedtakene de sammen overkriftene? Hva er de stabile strukturene vi kan basere oss på når vi skal parse teksten vedtakene?




In [18]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTTextLine, LTChar

def extract_text_with_font(pdf_path):
    text_info = []

    for page_layout in extract_pages(pdf_path):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                for text_line in element:
                    if isinstance(text_line, LTTextLine):
                        # Attempt to find the first character in the line
                        first_char = next((char for char in text_line if isinstance(char, LTChar)), None)
                        
                        if first_char is not None:
                            # Check if the first character is bold
                            is_bold = "Bold" in first_char.fontname

                            # If the first character is bold, consider the entire line bold
                            if is_bold:
                                text_info.append({
                                    "text": text_line.get_text(),  # Get the entire line text
                                    "font_name": first_char.fontname,
                                    "size": first_char.size,
                                    "is_bold": is_bold  # This key is optional but can be useful for clarity
                                })

    return text_info

def main(pdf_path):
    teller = 0
    text_info = extract_text_with_font(pdf_path)

    print("\nIdentifiserte tekstlinjer med uthevet tekst:")
    print("--------------------------------------------")
    for info in text_info:
        print(info["text"].strip())  # This will print each bold line as a separate string
        teller +=1
    print("\nTotalt", teller, ("overskrifter."))

if __name__ == "__main__":
    pdf_path = "./pdfs_small/2021-01885.pdf"  # replace with your PDF file path
    main(pdf_path)



Identifiserte tekstlinjer med uthevet tekst:
--------------------------------------------
Vedtak i Transportklagenemnda - Fly
Sammendrag
Dato
Saksnummer
Tjenesteytere
Klager har i det vesentlige anført
Tjenesteyterne har i det vesentlige anført
Nemnda bemerker
Vedtak
Nemndas representanter

Totalt 10 overskrifter.


Koden finner det vi forventer. Den har identifisert alle overskrifter og utelatt annen tekst. Dette var for et enkelt vedtak. For å undersøke videre om denne metoden er robust, tester vi på alle vedtak. Det gir ikke mening å printe ut alle overskrifter for 7827 vedtak. Vi leter derfor igjennom tekstlinjer med fet font og tar vare på alle unike tekstlinjer vi finner. Det skal da gi alle overskrifter som er brukt i vedtakene. Om metoden plukker med noe mer, vil vi også se det. 

In [21]:
import os
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTTextLine, LTChar

def extract_text_with_font(pdf_path):
    text_info = []

    for page_layout in extract_pages(pdf_path):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                for text_line in element:
                    if isinstance(text_line, LTTextLine):
                        # Attempt to find the first character in the line
                        first_char = next((char for char in text_line if isinstance(char, LTChar)), None)

                        if first_char is not None:
                            # Check if the first character is bold
                            is_bold = "Bold" in first_char.fontname

                            # If the first character is bold, consider the entire line bold
                            if is_bold:
                                text_info.append(text_line.get_text().strip())  # Get the entire line text

    return text_info

def main(directory_path):
    bold_lines_set = set()

    # Loop through every file in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            full_path = os.path.join(directory_path, filename)
            # Extract the bold lines from the current PDF and add them to the set
            bold_lines = extract_text_with_font(full_path)
            for line in bold_lines:
                bold_lines_set.add(line)

    print("\nIdentified lines with bold text:")
    print("--------------------------------------------")
    for line in bold_lines_set:
        print(line)  # This will print each unique bold line as a separate string

    print("\nTotal unique bold lines found:", len(bold_lines_set))

if __name__ == "__main__":
    directory_path = "./pdfs/"  # replace with your directory path
    main(directory_path)



Identified lines with bold text:
--------------------------------------------
Tjenesteyter følger ikke vedtaket i saken (30.03.2023)
Dato
Tjenesteyter følger ikke vedtaket i saken (28.09.2022)
Tjenesteytere
Tjenesteyter følger ikke vedtaket i saken (01.11.2022)
Vedtak i Transportklagenemnda - Fly
Tjenesteyter følger ikke vedtaket i saken (20.09.2019)
Tjenesteyter følger ikke vedtaket i saken (15.08.2023)
Vedtak i Pakkereisenemnda
Tjenesteyter følger ikke vedtaket i saken (21.12.2022)
Tjenesteyter følger ikke vedtaket i saken (16.04.2020)
Klager har i det vesentlige anført
Tjenesteyter følger ikke vedtaket i saken (11.10.2022)
Tjenesteyter følger ikke vedtaket i saken (10.01.2023)
Vedtak i Transportklagenemnda - Sjø
Tjenesteyter følger ikke vedtaket i saken (21.08.2017)
Tjenesteyter følger ikke vedtaket i saken (20.09.2022)
Tjenesteyter følger ikke vedtaket i saken (06.07.2023)
Sammendrag
Tjenesteyter følger ikke vedtaket i saken (13.03.2023)
Nemndas representanter
Tjenesteyter følger 

Resultatet er betryggende. Vi får et begrenset antall overskrifter. Mange av er variasjoner av samme med et tillegg av en dato. Her kan vi med tanke på parsingen fjerne datoen slik at alle disse blir like. Etter å ha fjernet variasjonen med dato, står vi igjen med 15 ulike overskrifter. Fire av disse er hovedoverskrifter på vedtakene:
- Vedtak i Pakkereisenemnda
- Vedtak i Transportklagenemnda - Fly
- Vedtak i Transportklagenemnda - Sjø
- Vedtak i Transportklagenemnda - Kollektivreiser
I pdf-en er overskriften til Transportklagenemnda - Kollektivreiser delt over to linjer. Derfor kommer den opp både som "Transportklagenemnda" og som "- Kollektivreiser".

For å vite om det er variasjon mellom vedtakene i bruken av overskrifter, dvs. om alle overskrifter er med i alle vedtak, kjører vi en sjekk også på dette. Vi lagrer da alle overskrifter for et vedtak i en liste og sjekker hvor mange forskellige lister vi får.

In [ ]:
import os
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTTextLine, LTChar
from collections import defaultdict

def extract_text_with_font(pdf_path):
    text_info = []

    for page_layout in extract_pages(pdf_path):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                for text_line in element:
                    if isinstance(text_line, LTTextLine):
                        first_char = next((char for char in text_line if isinstance(char, LTChar)), None)

                        if first_char is not None and "Bold" in first_char.fontname:
                            text_info.append(text_line.get_text().strip())

    return text_info

def main(directory_path):
    all_pdfs_bold_lines = []
    list_occurrences = defaultdict(int)

    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            full_path = os.path.join(directory_path, filename)
            print(f"Processing {filename}...")

            bold_lines = extract_text_with_font(full_path)
            if bold_lines:  # Only add non-empty lists
                # Convert list to a tuple so it can be hashed for the dictionary of occurrences
                bold_lines_tuple = tuple(bold_lines)
                all_pdfs_bold_lines.append(bold_lines_tuple)
                list_occurrences[bold_lines_tuple] += 1

    unique_lists = set(all_pdfs_bold_lines)

    print("\nNumber of unique lists: ", len(unique_lists))
    for unique_list in unique_lists:
        print("\nItems in the unique list:")
        for item in unique_list:
            print(item)
        print("Number of PDFs with this unique list: ", list_occurrences[unique_list])

if __name__ == "__main__":
    directory_path = "./pdfs"  # replace with your directory path
    main(directory_path)
